This notebook is a sandbox for playing with various ideas for Transcript Locations [[slides](https://docs.google.com/presentation/d/1sRKkO-9wdfNfDYonx17glXG0kH-39HZBwlstPIXvZTk/edit#slide=id.g9bf6c5914f_0_53)]. This notebook depends on an unpublished VRS schema in the [transcript-location branch of vr-spec](https://github.com/ga4gh/vr-spec/tree/199-transcript-locations) (as of 2020-10-07).

Two models are currently being considered:

* Unified model of transcripts (Reece).  A Transcript is merely a collection of exons and cds definition on any sequence.  For example, a RefSeq transcript on the defining RefSeq sequence would be one Transcript, and the projection of that transcript onto a genomic sequence (through splign, typicaly) would be a *different* Transcript with the *same* data structure.

* Annotated sequence model (Alex). The same Transcript model as above would be used only to define the transcript on the eponymous sequence. A separate AnnotedSequence model, with separate feature-based structures, would represent annotations of features on genomic sequences.


Considering HGVS syntax `NC_000007.13(NM_005228.4)` is useful. One interpretation of that syntax is that it is specifying the `NM_005228.4` exon structure on the `NC_000007.13` sequence. With that view, the AnnotatedSequence model uses distinct data structures for variants with reference sequences like `NM_005228.4` and `NC_000007.13(NM_005228.4)`. In contrast, the Unified Model effectively treats `NM_005228.4` as `NM_005228.4(NM_005228.4)`, and uses the same information model as for aligned sequences.

# Setup and Sample Data

In [1]:
from ga4gh.core import ga4gh_identify, ga4gh_serialize
from ga4gh.vrs import models, vr_enref, vr_deref

In [2]:
# Example transcript: NM_000314.6 (PTEN) on chr 10

# NM_000314.6(PTEN):c.78C>A  ~  NC_000010.11:g.87864547C>T
# NM_000314.6(PTEN):c.79+3A>G  ~  NC_000010.11:g.87864551A>G

# NM_000314.6
# These exons and CDS on the NM_000314.6 sequences constitute the
# definition of this transcript
t_exons = [(0,1110), (1110,1195), (1195,1240),
            (1240,1284), (1284,1523), (1523,1665),
            (1665,1832), (1832,2057), (2057,8701)]
t_cds = (1031,2243)

# NM_000314.6 aligned (by NCBI) to NC_000010.11 sequence (GRCh38 chr 10), + strand
g_exons = [(87863437, 87864548), (87894024, 87894109), (87925512, 87925557),
           (87931045, 87931089), (87933012, 87933251), (87952117, 87952259),
           (87957852, 87958019), (87960893, 87961118), (87965286, 87971930)]

# Cigars of alignment (relative to transcript)
tg_cigars = "666=1I39=1X404= 85= 45= 44= 239= 142= 167= 225= 6644="

# g_cds is computed from t_cds, accounting for alignment 
# 1032 = 1031 + 1I in cigar
g_cds = (87863437 + 1032, 87965286 + 2243 - 2057)


# Hardcode the ga4gh sequence identifier for NC_000010.11
t_sequence_id = 'ga4gh:SQ.7YNhHjHLiBJwNd43xjLJA7jjnuJwPhxn'
g_sequence_id = 'ga4gh:SQ.ss8r_wB0-b9r44TQTMmVTI92884QvBiB'

# In real implementations, do something like this:
#   from ga4gh.vrs.dataproxy import SeqRepoDataProxy
#   from biocommons.seqrepo import SeqRepo
#   dp = SeqRepoDataProxy(SeqRepo(root_dir="/usr/local/share/seqrepo/latest"))
#   t_sequence_id = dp.translate_sequence_identifier("refseq:NM_000314.6", "ga4gh")[0]
#   g_sequence_id = dp.translate_sequence_identifier("refseq:NC_000010.11", "ga4gh")[0]

# Defining a transcript on transcript reference sequence
That is, NM_000314.6 exon structure as defined on the NM_000314.6 transcript sequence.

The unified and annotated sequence models are identical in this regard.

In [3]:
t_transcript = models.Transcript(
  sequence_id = t_sequence_id,
  exons = [models.SimpleInterval(start=ex[0], end=ex[1]) for ex in t_exons],
  cds = models.SimpleInterval(start=t_cds[0], end=t_cds[1])
)

In [4]:
t_transcript.as_dict()

{'cds': {'end': 2243, 'start': 1031, 'type': 'SimpleInterval'},
 'exons': [{'end': 1110, 'start': 0, 'type': 'SimpleInterval'},
  {'end': 1195, 'start': 1110, 'type': 'SimpleInterval'},
  {'end': 1240, 'start': 1195, 'type': 'SimpleInterval'},
  {'end': 1284, 'start': 1240, 'type': 'SimpleInterval'},
  {'end': 1523, 'start': 1284, 'type': 'SimpleInterval'},
  {'end': 1665, 'start': 1523, 'type': 'SimpleInterval'},
  {'end': 1832, 'start': 1665, 'type': 'SimpleInterval'},
  {'end': 2057, 'start': 1832, 'type': 'SimpleInterval'},
  {'end': 8701, 'start': 2057, 'type': 'SimpleInterval'}],
 'sequence_id': 'ga4gh:SQ.7YNhHjHLiBJwNd43xjLJA7jjnuJwPhxn',
 'type': 'Transcript'}

In [5]:
ga4gh_identify(t_transcript)

'ga4gh:VTX.nTRjcOgzR6_owupcO39owUADNZeFY0d3'

# Defining a transcript on genomic references

In [6]:
# The Unified Model uses the same data structure as used above, but the underlying sequence and coordinates differ.
g_transcript = models.Transcript(
  sequence_id = g_sequence_id,
  exons = [models.SimpleInterval(start=ex[0], end=ex[1]) for ex in g_exons],
  cds = models.SimpleInterval(start=g_cds[0], end=g_cds[1])
)
g_transcript_id = ga4gh_identify(g_transcript)
g_transcript_id

'ga4gh:VTX.pTvbXNiGxYhmHFvCn7FeBnAFb14DzAZr'

## Comparison

In [7]:
# Both models have this definition of the transcript on the transcript sequence:
t_transcript.as_dict()

{'cds': {'end': 2243, 'start': 1031, 'type': 'SimpleInterval'},
 'exons': [{'end': 1110, 'start': 0, 'type': 'SimpleInterval'},
  {'end': 1195, 'start': 1110, 'type': 'SimpleInterval'},
  {'end': 1240, 'start': 1195, 'type': 'SimpleInterval'},
  {'end': 1284, 'start': 1240, 'type': 'SimpleInterval'},
  {'end': 1523, 'start': 1284, 'type': 'SimpleInterval'},
  {'end': 1665, 'start': 1523, 'type': 'SimpleInterval'},
  {'end': 1832, 'start': 1665, 'type': 'SimpleInterval'},
  {'end': 2057, 'start': 1832, 'type': 'SimpleInterval'},
  {'end': 8701, 'start': 2057, 'type': 'SimpleInterval'}],
 'sequence_id': 'ga4gh:SQ.7YNhHjHLiBJwNd43xjLJA7jjnuJwPhxn',
 'type': 'Transcript'}

In [8]:
# The Unified Model uses a similar structure on the genomic sequence
g_transcript.as_dict()

{'cds': {'end': 87965472, 'start': 87864469, 'type': 'SimpleInterval'},
 'exons': [{'end': 87864548, 'start': 87863437, 'type': 'SimpleInterval'},
  {'end': 87894109, 'start': 87894024, 'type': 'SimpleInterval'},
  {'end': 87925557, 'start': 87925512, 'type': 'SimpleInterval'},
  {'end': 87931089, 'start': 87931045, 'type': 'SimpleInterval'},
  {'end': 87933251, 'start': 87933012, 'type': 'SimpleInterval'},
  {'end': 87952259, 'start': 87952117, 'type': 'SimpleInterval'},
  {'end': 87958019, 'start': 87957852, 'type': 'SimpleInterval'},
  {'end': 87961118, 'start': 87960893, 'type': 'SimpleInterval'},
  {'end': 87971930, 'start': 87965286, 'type': 'SimpleInterval'}],
 'sequence_id': 'ga4gh:SQ.ss8r_wB0-b9r44TQTMmVTI92884QvBiB',
 'type': 'Transcript'}

In [9]:
# All of the above may be used to generated computed identfiers
# Note that the Unified Model uses the same prefix for transcripts regardless of underlying sequence
(ga4gh_identify(t_transcript), ga4gh_identify(g_transcript))

('ga4gh:VTX.nTRjcOgzR6_owupcO39owUADNZeFY0d3',
 'ga4gh:VTX.pTvbXNiGxYhmHFvCn7FeBnAFb14DzAZr')

# Defining transcript locations for CDS variation
NM_000314.6(PTEN):c.78C>A  ~  NC_000010.11:g.87864547C>T

CDS start c.1 corresponds to n.1032.
c.78 corresponds to n.1110, or interbase position 1109,1110 on sequence NM_000314.6.
Due to the 1 nt insertion on NC_000010.11, that position aligns to interbase position 1110,1111.

In [10]:
t_tloc = models.TranscriptLocation(
    transcript = t_transcript,
    interval = models.SimpleInterval(start=1109,end=1110))
t_tloc.as_dict()

{'interval': {'end': 1110, 'start': 1109, 'type': 'SimpleInterval'},
 'transcript': {'cds': {'end': 2243, 'start': 1031, 'type': 'SimpleInterval'},
  'exons': [{'end': 1110, 'start': 0, 'type': 'SimpleInterval'},
   {'end': 1195, 'start': 1110, 'type': 'SimpleInterval'},
   {'end': 1240, 'start': 1195, 'type': 'SimpleInterval'},
   {'end': 1284, 'start': 1240, 'type': 'SimpleInterval'},
   {'end': 1523, 'start': 1284, 'type': 'SimpleInterval'},
   {'end': 1665, 'start': 1523, 'type': 'SimpleInterval'},
   {'end': 1832, 'start': 1665, 'type': 'SimpleInterval'},
   {'end': 2057, 'start': 1832, 'type': 'SimpleInterval'},
   {'end': 8701, 'start': 2057, 'type': 'SimpleInterval'}],
  'sequence_id': 'ga4gh:SQ.7YNhHjHLiBJwNd43xjLJA7jjnuJwPhxn',
  'type': 'Transcript'},
 'type': 'TranscriptLocation'}

In [11]:
# Unified Model
g_tloc = models.TranscriptLocation(
    transcript = g_transcript,
    interval = models.SimpleInterval(start=1110,end=1111))
g_tloc.as_dict()

{'interval': {'end': 1111, 'start': 1110, 'type': 'SimpleInterval'},
 'transcript': {'cds': {'end': 87965472,
   'start': 87864469,
   'type': 'SimpleInterval'},
  'exons': [{'end': 87864548, 'start': 87863437, 'type': 'SimpleInterval'},
   {'end': 87894109, 'start': 87894024, 'type': 'SimpleInterval'},
   {'end': 87925557, 'start': 87925512, 'type': 'SimpleInterval'},
   {'end': 87931089, 'start': 87931045, 'type': 'SimpleInterval'},
   {'end': 87933251, 'start': 87933012, 'type': 'SimpleInterval'},
   {'end': 87952259, 'start': 87952117, 'type': 'SimpleInterval'},
   {'end': 87958019, 'start': 87957852, 'type': 'SimpleInterval'},
   {'end': 87961118, 'start': 87960893, 'type': 'SimpleInterval'},
   {'end': 87971930, 'start': 87965286, 'type': 'SimpleInterval'}],
  'sequence_id': 'ga4gh:SQ.ss8r_wB0-b9r44TQTMmVTI92884QvBiB',
  'type': 'Transcript'},
 'type': 'TranscriptLocation'}

## Transcript Allele

In [12]:
a = models.Allele(
    location = g_tloc,
    state = models.SequenceState(sequence="C"))

## Transcript Feature Locations

In [13]:
tf = models.TranscriptFeature(feature_type="exon", index=0)
tf.as_dict()

{'feature_type': 'exon', 'index': 0}

In [14]:
tfi = models.TranscriptFeatureInterval(
    start=models.TranscriptFeature(feature_type="exon", index=0),
    end=models.TranscriptFeature(feature_type="exon", index=5),
)
tfi.as_dict()

{'end': {'feature_type': 'exon', 'index': 5},
 'start': {'feature_type': 'exon', 'index': 0},
 'type': 'TranscriptFeatureInterval'}

In [15]:
tfl = models.TranscriptLocation(
    transcript = g_transcript,
    interval = tfi)
tfl.as_dict()

{'interval': {'end': {'feature_type': 'exon', 'index': 5},
  'start': {'feature_type': 'exon', 'index': 0},
  'type': 'TranscriptFeatureInterval'},
 'transcript': {'cds': {'end': 87965472,
   'start': 87864469,
   'type': 'SimpleInterval'},
  'exons': [{'end': 87864548, 'start': 87863437, 'type': 'SimpleInterval'},
   {'end': 87894109, 'start': 87894024, 'type': 'SimpleInterval'},
   {'end': 87925557, 'start': 87925512, 'type': 'SimpleInterval'},
   {'end': 87931089, 'start': 87931045, 'type': 'SimpleInterval'},
   {'end': 87933251, 'start': 87933012, 'type': 'SimpleInterval'},
   {'end': 87952259, 'start': 87952117, 'type': 'SimpleInterval'},
   {'end': 87958019, 'start': 87957852, 'type': 'SimpleInterval'},
   {'end': 87961118, 'start': 87960893, 'type': 'SimpleInterval'},
   {'end': 87971930, 'start': 87965286, 'type': 'SimpleInterval'}],
  'sequence_id': 'ga4gh:SQ.ss8r_wB0-b9r44TQTMmVTI92884QvBiB',
  'type': 'Transcript'},
 'type': 'TranscriptLocation'}

In [16]:
ga4gh_identify(tfl)

'ga4gh:VTL.h2wLFU5g0cp5AxX59QsVKcllRljBhF-b'

In [17]:
vr_enref(tfl).as_dict()

{'interval': {'end': {'feature_type': 'exon', 'index': 5},
  'start': {'feature_type': 'exon', 'index': 0},
  'type': 'TranscriptFeatureInterval'},
 'transcript': 'ga4gh:VTX.pTvbXNiGxYhmHFvCn7FeBnAFb14DzAZr',
 'type': 'TranscriptLocation'}